In [40]:
#Load in utility classes
%run ~/projects/testlipids/testlipids.ipynb
import os #Operating system specific commands
import re #Regular expression library

membrane="epithelial"
insane="./insane+SF.py"
mdparams="./test.mdp"
martinipath="./martini.ff"

In [41]:
# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}.gro
!rm -f {membrane}.edr
!rm -f {membrane}.log
!rm -f {membrane}.pdb
!rm -f {membrane}.tpr
!rm -f {membrane}.trr

In [42]:
cmdline = "-u OIPE:0.002 -u PQPE:0.002 -u OQPE:0.001 -u OAPE:0.002 -u OUPE:0.001 -u PBSM:0.007 -u PXSM:0.004 -u DODG:0.004 -u DPPMC:0.007 -u POPMC:0.043 -u DOPMC:0.009 -u PIPMC:0.009 -u OIPMC:0.004 -u OEPMC:0.005 -u PAPMC:0.012 -u PUPMC:0.029 -u POPME:0.001 -u DOPME:0.006 -u OIPME:0.001 -u OQPME:0.002 -u OAPME:0.014 -u OUPME:0.006 -u IAPME:0.007 -u IQPME:0.001 -u LPPC:0.008 -u DPPC:0.003 -u POPC:0.147 -u DOPC:0.011 -u PIPC:0.088 -u OIPC:0.05 -u PAPC:0.052 -u PUPC:0.05 -u POPE:0.007 -u OPPE:0.002 -u DOPE:0.007 -u PAPE:0.006 -u PUPE:0.002 -u POPG:0.004 -u DOPG:0.003 -u DPSM:0.017 -u PNSM:0.008 -u PODG:0.007 -l OIPE:0.008 -l PQPE:0.006 -l OQPE:0.004 -l OAPE:0.009 -l OUPE:0.005 -l OGPS:0.004 -l DOPI:0.008 -l PEPI:0.004 -l PQPI:0.039 -l PBSM:0.003 -l PXSM:0.002 -l DODG:0.002 -l DPPMC:0.003 -l POPMC:0.016 -l DOPMC:0.003 -l PIPMC:0.003 -l OIPMC:0.002 -l OEPMC:0.002 -l PAPMC:0.004 -l PUPMC:0.011 -l POPME:0.005 -l DOPME:0.02 -l OIPME:0.003 -l OQPME:0.006 -l OAPME:0.052 -l OUPME:0.023 -l IAPME:0.024 -l IQPME:0.003 -l PAPS:0.004 -l LPPC:0.003 -l DPPC:0.001 -l POPC:0.056 -l DOPC:0.004 -l PIPC:0.034 -l OIPC:0.019 -l PAPC:0.02 -l PUPC:0.019 -l POPA:0.01 -l DOPA:0.005 -l POPE:0.025 -l OPPE:0.008 -l DOPE:0.026 -l PAPE:0.02 -l PUPE:0.007 -l POPG:0.003 -l DOPG:0.002 -l DPPS:0.001 -l POPS:0.093 -l DOPS:0.006 -l POPI:0.025 -l PIPI:0.006 -l PAPI:0.055 -l PUPI:0.008 -l DPSM:0.007 -l PNSM:0.003 -l PODG:0.004 "
 

In [43]:
print("Build")
build = !python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 16.6 -y 19.4 -z 16 -sol PW -salt 0.15 -center -charge 0 -orient {cmdline}  
for line in build:
    print(line)


Build
; X: 16.600 (21 lipids) Y: 19.400 (25 lipids)
; 525 lipids in upper leaflet, 525 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 12540
; Charge of membrane: -77.000000
; Total charge: -77.000000
; NDX Solvent 12541 110257
; NDX System 1 110257


In [56]:
# remove extra descriptive lines in the gro file that would otherwise overrun the buffer during GROMPP
import datetime
generationDate = datetime.datetime.now().strftime("%Y.%m.%d")
!sed -i '1s/.*/Complex Epithelial membrane generated ({generationDate}) using INSANE/' {membrane}.gro

In [49]:

print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("Fatal error", line):
        success=False
    #if not success:
    print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)


Grompp
                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GRO

In [50]:
# test lipids
#Load in utility classes
%run ./testlipids.ipynb

#load lipids from GRO file
lipids=lipidsfromsystem("./epithelial")
print(lipids)

testlipids(lipids,membrane,insane,mdparams,martinipath).execute(True).report()

Converting ./epithelial.gro to a pdb file
using system specified in ./epithelial.pdb file
['OIPE', 'PQPE', 'OAPE', 'PBSM', 'PXSM', 'DODG', 'DPPM', 'POPM', 'DOPM', 'PIPM', 'OIPM', 'OEPM', 'PAPM', 'PUPM', 'OQPM', 'OAPM', 'OUPM', 'IAPM', 'LPPC', 'DPPC', 'POPC', 'DOPC', 'PIPC', 'OIPC', 'PAPC', 'PUPC', 'POPE', 'OPPE', 'DOPE', 'PAPE', 'PUPE', 'POPG', 'DOPG', 'DPSM', 'PNSM', 'PODG', 'OQPE', 'OUPE', 'OGPS', 'DOPI', 'PEPI', 'PQPI', 'IQPM', 'PAPS', 'POPA', 'DOPA', 'POPS', 'DOPS', 'POPI', 'PIPI', 'PAPI', 'PUPI']
[52]testing lipid OIPE in membrane epithelial...create...compile...minimize=[fail]: 7.07 secs
[51]testing lipid PQPE in membrane epithelial...create...compile...minimize=[success]: 15.86 secs
[50]testing lipid OAPE in membrane epithelial...create...compile...minimize=[success]: 18.81 secs
[49]testing lipid PBSM in membrane epithelial...create...compile...minimize=[success]: 19.12 secs
[48]testing lipid PXSM in membrane epithelial...create...compile...minimize=[success]: 18.94 secs
[47]tes

In [53]:
# test lipids
#Load in utility classes
%run ./testlipids.ipynb

# test an individual lipis species
import os #Operating system specific commands
#re-compile
lipid = 'DPPM'
results=[]

if build(os.getcwd(),membrane,lipid,insane).execute().report(results).success:
    comp = compile(os.getcwd(),membrane,lipid,mdparams,martinipath).execute().report(results)
    print(lipid)
    if not comp.success:
        for line in comp.output:
            print(line)
else:
    for line in results:
        print(line)
print("done")

['DPPM', False, ['; X: 27.000 (35 lipids) Y: 27.000 (35 lipids)', '; 1225 lipids in upper leaflet, 1225 lipids in lower leaflet', 'Traceback (most recent call last):', '  File "/home/richard/projects/epithelial/insane+SF.py", line 1321, in <module>', '    atoms    = zip(lipidsa[lipid][1].split(),lipidsx[lipidsa[lipid][0]],lipidsy[lipidsa[lipid][0]],lipidsz[lipidsa[lipid][0]])', "KeyError: 'DPPM'"]]
done
